In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

#  "Scraping" los datos desde wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "lxml")

## Limpieza de los datos

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

## Checando datos no asignados

In [4]:
print(f"There are {df.loc[df['Borough'] == 'Not assigned'].shape[0]} Not assigned Boroughs")

There are 0 Not assigned Boroughs


In [5]:
print(f"There are {df.loc[df['Neighborhood'] == 'Not assigned'].shape[0]} Not assigned Neighborhoods")

There are 0 Not assigned Neighborhoods


In [6]:

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Checando si el ejemplo converge con los datos

In [7]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

df.loc[df['PostalCode'].isin(example_postalcode)]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


## Imprimiendo la forma de los datos

In [8]:
df.shape

(103, 3)

# Obteniendo las coordenadas geográficas a partir de los datos

In [9]:
import geocoder

for index, content in df.iterrows():
    coordinates = None
    while(coordinates is None):
        coordinates = geocoder.arcgis(f'{content[0]}, Toronto, Ontario').latlng
    latitude = coordinates[0]
    longitude = coordinates[1]
    df.loc[index, 'Latitude'] = latitude
    df.loc[index, 'Longitude'] = longitude

In [13]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753568,-79.329231
1,M4A,North York,Victoria Village,43.727295,-79.313077
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655208,-79.363013
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722074,-79.450524
4,M7A,Queen's Park,Ontario Provincial Government,43.662300,-79.389293
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.654818,-79.509160
99,M4Y,Downtown Toronto,Church and Wellesley,43.666461,-79.381366
100,M7Y,East Toronto Business,Enclave of M4L,43.662094,-79.322488
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.633107,-79.494667


In [11]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df = df.merge(coordinates.set_index('PostalCode'), on='PostalCode')

In [20]:
df.drop(['Latitude_y','Longitude_y'],axis=1, inplace = True)

In [23]:
df.rename(columns = {'Latitude_x':'Latitude','Longitude_x':'Longitude'}, inplace=True)

## Checando si el ejemplo converge con los datos

In [24]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

df.loc[df['PostalCode'].isin(example_postalcode)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655208,-79.363013
6,M1B,Scarborough,"Malvern, Rouge",43.809240,-79.193050
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706446,-79.309987
23,M4G,East York,Leaside,43.708840,-79.366144
24,M5G,Downtown Toronto,Central Bay Street,43.657056,-79.386153
27,M2H,North York,Hillcrest Village,43.801706,-79.358258
32,M1J,Scarborough,Scarborough Village,43.746641,-79.231962
50,M9L,North York,Humber Summit,43.759314,-79.556453
54,M4M,East Toronto,Studio District,43.658330,-79.345685
71,M1R,Scarborough,"Wexford, Maryvale",43.751328,-79.301072


# Explore y agrupe los vecindarios en Toronto

In [25]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [33]:
df_toronto_borough = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_borough.shape

(39, 5)

In [34]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_borough['Latitude'], df_toronto_borough['Longitude'], df_toronto_borough['Borough'], df_toronto_borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [35]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [36]:
df_toronto_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655208,-79.363013
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657808,-79.378409
2,M5C,Downtown Toronto,St. James Town,43.651503,-79.375763
3,M4E,East Toronto,The Beaches,43.678223,-79.293751
4,M5E,Downtown Toronto,Berczy Park,43.645428,-79.373288
